In [10]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [ ]:
from pathlib import Path

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

##change everytime
extracted_raw_file = Path(r"work_on_minutes\acta_02_08_2024\extracted_text_02_08_24_clean_vers_001.txt")

text_content = []

with open(extracted_raw_file, "r", encoding="utf-8") as text_file:
    text_full = text_file.read()
    
    # Count the number of tokens    
    text_tokens = encoding.encode(text_full)
    num_tokens = len(text_tokens)
    print(f"Number of tokens : {num_tokens}")

    text_content.append(text_full) #save for future processing
     
    print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

In [12]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

### Prompt Engineering 1

In [ ]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = f"Has un resumen de una minuta de reunión de un gobierno local.\
La intención de este ejercicio es proveer al ciudadano común con información relevante sobre el desempeño de su gobierno local. \
Instrucciones especificas: \
- Enfatiza los problemas, peticiones y soluciones o acuerdos que se llegan \
- Indica los nombres de las personas que participan cuando sea necesario.\
- Indica datos concretos \
- No emitas juicios de valor, manten neutralidad\
- Identifica fechas\
- Identifica los temas que se trataron.\
- Verifica con cuidado al final cuantas personas asistieron a la sesion\
"

# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


### Prompt Engineering 2

In [43]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = """Has un resumen de una minuta de reunión de un gobierno local con las siguientes instrucciones:
-  Enfatiza Los principales temas que se hablaron en la sesion, enfatizando los problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan cuando sea necesario.
    - Provee datos concretos
    - No emitas juicios de valor
    - Verifica con cuidado cuantas personas asistieron a la sesion"
    
    - Envia la informacion con la siguiente estructura:
     {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_numero_del_acta", 
     "act_name": "nombre del acta", 
     "content:  [" resumen de la sesion en donde se explique de manera fluida y clara con las instrucciones enviadas arriba"], 
    } 
    """


# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


Number of tokens: 181


In [33]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]

In [44]:

client = OpenAI(api_key = key_ct)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.80,
    max_tokens = 1200,
)


In [45]:
#change everytime
response_summary_TOTAL_vers001 = response.choices[0].message.content
print(response_summary_TOTAL_vers001)   

{
  "date": "2024-08-02",
  "document_id": "acta_077",
  "act_name": "Sesión Conmemorativa de Concejo No. 077",
  "content": [
    "La Sesión Conmemorativa de Concejo No. 077 se llevó a cabo el 2 de agosto de 2024, a las 10:17, con la asistencia de 16 concejales presentes, constatando el cuórum necesario para su instalación. La sesión fue dirigida por el Alcalde del Distrito Metropolitano de Quito, Pabel Muñoz López, y tuvo como principal motivo la conmemoración de la \"Matanza de los Próceres del Primer Grito de Independencia\". ",
    "Durante la sesión, se destacó el discurso de la Concejala Metropolitana, Joselyn Mayorga Salazar, quien enfatizó la importancia de recordar la participación de las mujeres en el proceso independentista y la lucha por la justicia y la libertad. Mayorga subrayó que, a pesar de los intentos de invisibilizar la historia de estas mujeres, su legado sigue vigente en la lucha actual por los derechos. Hizo un llamado a la unidad y al trabajo colectivo en la po

### Formatting

In [ ]:
#extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_TOTAL_vers001_formatted = extract_markdown_content(response_summary_TOTAL_vers001)
print(response_summary_TOTAL_vers001_formatted)


## Exporting to files

### Exporting to .txt 

In [46]:

#change everytime
with open(r"work_on_minutes\acta_02_08_2024\response_summary_TOTAL_vers002.json", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers001)  # Write the block text to the file, with spacing between blocks
    #review the response everytime

print("Text extraction completed and saved'.")


Text extraction completed and saved'.


### Exporting to .md

In [ ]:

#change everytime
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\work_on_minutes\acta_26_abril_2019\summary_26_04_19\response_summary_TOTAL_vers002.md", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers002)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved to 'response_summary_TOTAL_vers002'.")


### Exporting to .json

In [ ]:

# Write the JSON data to a new file
with open(r"work_on_minutes\acta_05_diciembre_2023\response_summary_TOTAL_vers002.json", "w", encoding="utf-8") as json_file:
    json.dump( response_summary_TOTAL_vers002_formatted, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")